# Redes Neuronales - TP2
## Ej 4
- a - Implemente una red con aprendizaje Backpropagation que aprenda la siguiente
función:

$$f (x , y , z)=sin(x)+cos(y)+z$$

donde: x e y ∊ [0,2 π] y z ∊[−1,1]. Para ello construya un conjunto de datos de entrenamiento y un conjunto de evaluación. Muestre la evolución del error  e
entrenamiento y de evaluación en función de las épocas de entrenamiento.

- b - Estudie la evolución de los errores durante el entrenamiento de una red con una capa oculta de 30 neuronas cuando el conjunto de entrenamiento contiene 40 muestras. ¿Que ocurre si el minibatch tiene tamaño 40? ¿Y si tiene tamaño 1?

Acá la consigna lo lo explicita pero hay que aplicar algo de minibatch al lagoritmo de aprendizaje.

Robando de un post de stackExchange:

batch gradient descent you process the entire training set in one iteration. Whereas, in a mini-batch gradient descent you process a small subset of the training set in each iteration.

Also compare stochastic gradient descent, where you process a single example from the training set in each iteration.

Another way to look at it: they are all examples of the same approach to gradient descent with a batch size of m and a training set of size n. For stochastic gradient descent, m=1. For batch gradient descent, m = n. For mini-batch, m=b and b < n, typically b is small compared to n.

Mini-batch adds the question of determining the right size for b, but finding the right b may greatly improve your results.




Dicho todo esto, la idea es tomar el set de datos y generar batches de entrenamiento. Es otro loop for y una partición de los datos de entrada. hay 2 hiperparams -> la # de minibatches y la # de veces que hacemos el proceso. Para mantener la cantidad de muestras balanceadas, solo se le pueden mostrar 1 vez por batch de minibatch

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

Vamos a generar las muestras de la función. Voy a barrer X, Y de 0 a 2pi para Z=+-1

In [38]:
x = np.linspace(0,1,num=200)*2*np.pi
y = x # hasta ahora tenemos los ejes
z = [-1,1]
f = []
tripletas_xyz = []

for k in z:
    for i in x:
        for j in y:
            tripletas_xyz.append([i,j,k])

    
# este es el dataset entero, es gigante. 

# -> lo pasamos a matriz

dataset = np.asmatrix(tripletas_xyz) # listo. 

# no lo shuffleo acá, es lo va a hacer el entrenador
# voy a generar el f

f = np.sin(dataset[:,0])+np.cos(dataset[:,1])+dataset[:,2]




Ahora toca shufflear las filas y hacer dataset de entrenamiento y de testeo. Por comodidad lo hago con sklearn, el resto si será a mano. Como tengo 80mil registros, y quiero batches de números enteros, voy a separar esto en una proporción 70% 30%, que me parece razonable. Luego, los minibatches serán de números como 1,10,100,1000,2000,4000. 

In [39]:
X_train, X_test, y_train, y_test = train_test_split(  dataset, f, train_size = 56000, random_state=0)

In [ ]:
class perceptron_multicapa:
    def __init__(self, capas, dim_entrada):
        self.capas = capas
        self.dim_entrada = dim_entrada
        self.lista_matrices = []
        self.lr = None
        entrada_anterior = dim_entrada
        for num_perceptrones in capas:
            matriz_pesos = np.random.uniform(-2, 2, size=(num_perceptrones, entrada_anterior + 1)) # inicialización más amplia
            self.lista_matrices.append(matriz_pesos)
            entrada_anterior = num_perceptrones

    def funcion_activacion(self, x):
        return 1 / (1 + np.exp(-x))
    
    def derivada_activacion(self, a):
        return a * (1 - a)

    def forward(self, x):
        a = x
        activaciones = [a] # todos los x de la red cuando se propaga para adelante - es el zs pasado por la función de activación
        zs = []# todos los h de la red cuando se propaga para adelante
        for W in self.lista_matrices:
            a_b = np.concatenate(([1], a))
            z = np.dot(W, a_b)
            zs.append(z)
            a = self.funcion_activacion(z)
            activaciones.append(a)
        return activaciones, zs
    
    def predecir(self, X):
        salidas = []
        for x in X:
            a = x
            for W in self.lista_matrices:
                a = np.concatenate(([1], a))
                a = self.funcion_activacion(np.dot(W, a))
            salidas.append(a.reshape(-1))  # garantiza que sea vector 1D
        return np.array(salidas).reshape(len(X), -1)

    def predecir_hard(self, X):
        salidas = self.predecir(X)
        # Si la salida es entre 0 y 1 (sigmoide), umbral en 0.5
        return (salidas >= 0.5).astype(int)

    def entrenar(self, X, Y, lr=1.0, epochs=20000):
        self.lr = lr
        n_samples = X.shape[0]
        self.error_por_epoch = []
        for epoch in range(epochs):
            grad_acum = [np.zeros_like(W) for W in self.lista_matrices]
            for i in range(n_samples):
                x = X[i]
                y = Y[i]
                activaciones, zs = self.forward(x) # tomo los h y x del paso foward
                deltas = [None] * len(self.lista_matrices) # lugar para guardar los deltas
                a_out = activaciones[-1] # la activavión de la salida es la última
                deltas[-1] = (a_out - y) * self.derivada_activacion(a_out) # el último delta es el de la salida, y se calcula como la salida deseada - la actual por la derivada de la func de act. 
                for l in range(len(self.lista_matrices)-2, -1, -1): # acá propagamos desde la salida para aras
                    W_next = self.lista_matrices[l+1][:,1:]
                    delta_next = deltas[l+1]
                    a_l = activaciones[l+1]
                    deltas[l] = np.dot(W_next.T, delta_next) * self.derivada_activacion(a_l)
                for l in range(len(self.lista_matrices)):
                    a_prev = np.concatenate(([1], activaciones[l]))
                    grad_acum[l] += np.outer(deltas[l], a_prev)
            for l in range(len(self.lista_matrices)):
                self.lista_matrices[l] -= self.lr * grad_acum[l] / n_samples
            if epoch % 100 == 0 or epoch == epochs-1:
                pred = self.predecir(X)
                loss = np.mean((pred - Y) ** 2) # acá se va guardando el ECM para luego visualizarlo, pero se hace cada tanto, no los miles de puntos. 
                self.error_por_epoch.append(loss)

    def minibatch_training(self, X, Y, tamanio, veces, lr=0.1, epochs=20000):
        # X: matriz de entradas, Y: vector columna de targets
        n_samples = X.shape[0]
        for _ in range(veces):
            indices = np.random.permutation(n_samples)
            X_shuffled = X[indices]
            Y_shuffled = Y[indices]
            for start in range(0, n_samples, tamanio):
                end = min(start + tamanio, n_samples)
                X_batch = X_shuffled[start:end]
                Y_batch = Y_shuffled[start:end]
                self.entrenar(X_batch, Y_batch, lr=lr, epochs=epochs)

In [41]:
# datos para la XOR de 2 entradas y 1 salida (corregido)
A = np.array([0, 0, 1, 1])
B = np.array([0, 1, 0, 1])
Y12 = np.array([0, 1, 1, 0])
datos_XOR2 = np.column_stack((A, B, Y12))


In [43]:
# ahora toca entrenar el perceptron con estos datos
test2 = perceptron_multicapa(capas=[4,1], dim_entrada=2) # para la XOR de 2 entradas y 1 salida
test4 = perceptron_multicapa(capas=[4, 1], dim_entrada=4) # para la XOR de 4 entradas y 1 salida

# entrenamos ambos modelos con sus respectivos datos y targets 
X2 = datos_XOR2[:, 0:2]
Y2 = datos_XOR2[:, 2].reshape(-1, 1)  # targets 0/1 para sigmoide


test2.minibatch_training(X2, Y2, tamanio = 1, veces = 10, lr=0.1, epochs=20000)


IndexError: index 3 is out of bounds for axis 1 with size 3

In [ ]:
# Graficar frontera de decisión para XOR de 2 entradas
h = 0.01
x_min, x_max = X2[:,0].min() - 0.1, X2[:,0].max() + 0.1
y_min, y_max = X2[:,1].min() - 0.1, X2[:,1].max() + 0.1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

Z = test2.predecir(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, levels=[0,0.5,1], alpha=0.3, cmap="bwr")
plt.scatter(X2[:,0], X2[:,1], c=Y2[:,0], cmap="bwr", edgecolors="k")
plt.title("Frontera de decisión XOR (2 entradas)")
plt.show()


In [ ]:
# Mostrar predicciones hard y soft para los 4 puntos de la XOR de 2 entradas
print("Entradas\tTarget\tSalida sigmoide\tPredicción (umbral 0.5)")
preds = test2.predecir(X2)
for x, y, p in zip(X2, Y2, preds):
    print(f"{x}\t{int(y[0])}\t{p[0]:.3f}\t\t{int(p[0]>=0.5)}")

In [ ]:
def graficar_error(modelo, titulo):
    plt.figure()
    plt.plot(np.arange(len(modelo.error_por_epoch)) * 100, modelo.error_por_epoch, marker='o')
    plt.xlabel('Época')
    plt.ylabel('ECM')
    plt.title(titulo)
    plt.grid(True)
    plt.show()

# Graficar error para XOR 2 entradas
graficar_error(test2, "Evolución del error (ECM) - XOR 2 entradas")


la xor de 4 cae más rápido xq tiene 16 muestras para 4 dimensiones, mucbo menos proporiconalmente que 4 muestras para 2. diría que está más vacio el espacio R4 que el R2. 